In [1]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # active IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler




In [2]:
# 1. Charger les données
df = pd.read_csv("C:/Users/bamoi/OneDrive - Groupe ESAIP/$ PSIR/experimentation/repo/psir-ml-imputation/notebooks/hta_nhanes_enrichi.csv")



In [3]:
# 2. Séparer la colonne cible
target = df["HTA"]
df = df.drop(columns=["HTA"])



In [4]:
# 3. Séparer les colonnes numériques à imputer
colonnes_num = df.select_dtypes(include=["float64", "int64"]).columns
colonnes_non_num = df.select_dtypes(exclude=["float64", "int64"]).columns



In [5]:
# 4. Normalisation (important pour les algos de régression dans MICE)
scaler = StandardScaler()
df_num_scaled = pd.DataFrame(scaler.fit_transform(df[colonnes_num]), columns=colonnes_num)



In [6]:
# 5. Appliquer IterativeImputer (MICE)
imputer = IterativeImputer(max_iter=10, random_state=0)
df_num_imputed = pd.DataFrame(imputer.fit_transform(df_num_scaled), columns=colonnes_num)



c:\Users\bamoi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [7]:
# 6. Dé-normaliser les données
df_num_restored = pd.DataFrame(scaler.inverse_transform(df_num_imputed), columns=colonnes_num)



In [8]:
# 7. Reconstruire le dataset complet
df_non_num = df[colonnes_non_num].reset_index(drop=True)
df_imputed = pd.concat([df_non_num, df_num_restored], axis=1)



In [9]:
# 8. Réintégrer la cible HTA
df_imputed["HTA"] = target.values



In [10]:
# 9. Sauvegarder
df_imputed.to_csv("hta_impute_MICE.csv", index=False)
print("✅ Imputation MICE terminée. Dataset sauvegardé sous : hta_impute_MICE.csv")

✅ Imputation MICE terminée. Dataset sauvegardé sous : hta_impute_MICE.csv
